In [1]:
import numpy as np

To get Initial Matrix $B,N,C_N,C_B$

In [2]:
def get_intital(A,C,Initial_col):
    m,n = A.shape
    selected_col = Initial_col
    Not_selected_col  = np.setdiff1d(np.linspace(0,n-1,n) , Initial_col).astype(int)
    
    #B0 = A[:,selected_col].copy()
    #N0 = A[:,Not_selected_col].copy()
    
    B0 = np.squeeze(np.array([A[:,i] for i in Initial_col])).T
    N0 = np.squeeze(np.array([A[:,i] for i in Not_selected_col])).T
    
    C_b0 = np.squeeze(np.array([C.T[:,i] for i in Initial_col])).T
    C_n0 = np.squeeze(np.array([C.T[:,i] for i in Not_selected_col])).T
    
    #print(B0,N0,C_b0,C_n0)
    
    return B0,N0,C_b0,C_n0

For Z

In [3]:
def get_z(B,C_b):
    z = np.linalg.solve(B.T,C_b)
    return z

For r

In [4]:
def get_r(C_n,N,z):
    r = C_n - np.dot(N.T,z)
    return r

To calculate leaving variable

In [5]:
def leaving_var(r):
    res = np.argmin(r)
    return res

To calculate entering variable

In [378]:
def enterning_var(B,b,N,i,m):
    x_b = np.linalg.solve(B,b).reshape(-1,1)
    w = np.linalg.solve(B,N[:,i].copy()).reshape(-1,1)
    #print("X_b : ",x_b)
    #print("W : ",w)
    
    
    temp = np.array([0]*len(N)).astype(float)
    temp = np.divide(x_b,w)
    
    # to calculate the least positive value of x_b/w
    temp = np.array(list(zip(temp,[i for i in range(len(temp))])))
    temp = temp[temp[:,0]>0]
    temp = temp[temp[:,0].argsort()]
    res = temp[0][1]
    return int(res)

Final Solver

In [379]:
def Solver(A,C,b,Initial_col):
    
    m,n = A.shape
    Counts = True
    Track_col = []
    Track_temp = [j for j in range(n)]
    #Get Initials
    B,N,C_b,C_n = get_intital(A,C,Initial_col)
    Iterations = 0
    
    while(Counts):
        z = get_z(B,C_b)
        r = get_r(C_n,N,z)
        if(np.all(r>0)):
            Counts = False
            break
        #Calculate leaving and entering variable
        i = leaving_var(r)
        k = enterning_var(B,b,N,i,m)
        
        #Swap columns in B and N
        temp = 0
        temp = B[:,k].copy()
        B[:,k] = N[:,i]
        N[:,i] = temp
        
        #Swap element in C
        temp1 = 0
        temp1 = C_b[k].copy()
        C_b[k] = C_n[i]
        C_n[i] = temp1
                
        #keep track of columns
        temp2 = Track_temp[i+m]
        Track_temp[i+m] = Track_temp[k]
        Track_temp[k] = temp2
        Track_col.append(Track_temp)
        Iterations = Iterations + 1
        print("For " +str(Iterations)+"th Iteration B :\n",B, "\n")
    
        print("For " +str(Iterations)+"th Iteration N :\n",N,"\n")

        print("For " +str(Iterations)+"th Iteration r :\n",r,"\n")
    
    
    #Fianl Answer
    X_B = np.linalg.solve(B,b)
    Final_X = [0]*(n)
    if(len(Track_col)>0): 
        for index,value in enumerate(X_B):
            Final_X[Track_col[-1][index]] = value.item(0)
        Final_X = np.array(Final_X).reshape(-1,1)
    else:
        for index,value in enumerate(Initial_col):
            Final_X[value] = X_B[index].item(0)
    
    cost = np.dot(C.T,Final_X)
    cost = cost.item(0)
    print("Value of X : \n",Final_X,"\n")
    print("Cost : \n",cost,"\n")
    return Final_X

## Problem 1,Part 1

In [372]:
A = np.array([[4,-3,-1,4,1],
             [-1,2,2,1,3]])

C = np.array([10,-5,-4,7,1]).reshape(-1,1)

b = np.array([1,4]).reshape(-1,1)

In [373]:
Initial_col = np.array([0,1])
Result = Solver(A,C,b,Initial_col)

For 1th Iteration B :
 [[ 1 -3]
 [ 3  2]] 

For 1th Iteration N :
 [[-1  4  4]
 [ 2  1 -1]] 

For 1th Iteration r :
 [-5. -7. -8.] 

For 2th Iteration B :
 [[ 1 -1]
 [ 3  2]] 

For 2th Iteration N :
 [[-3  4  4]
 [ 2  1 -1]] 

For 2th Iteration r :
 [-2.09090909  1.          3.63636364] 

For 3th Iteration B :
 [[ 4 -1]
 [ 1  2]] 

For 3th Iteration N :
 [[-3  1  4]
 [ 2  3 -1]] 

For 3th Iteration r :
 [ 4.6 -3.6 -1.8] 

Value of X : 
 [[0.        ]
 [0.        ]
 [1.66666667]
 [0.66666667]
 [0.        ]] 

Cost : 
 -2.0 



## Problem 1,Part 2

In [374]:
A1 = np.array([[2,3,-2,1,2],
               [-2,1,0,2,3]])
C1 = np.array([4,2,1,4,-2]).reshape(-1,1)
b1 = np.array ([4,2]).reshape(-1,1)

In [375]:
Result1 = Solver(A1,C1,b1,Initial_col)

For 1th Iteration B :
 [[2 3]
 [3 1]] 

For 1th Iteration N :
 [[-2  1  2]
 [ 0  2 -2]] 

For 1th Iteration r :
 [ 3.  5. -1.] 

For 2th Iteration B :
 [[ 2  2]
 [ 3 -2]] 

For 2th Iteration N :
 [[-2  1  3]
 [ 0  2  1]] 

For 2th Iteration r :
 [ 3.28571429  5.71428571 -1.14285714] 

Value of X : 
 [[0.8]
 [0. ]
 [0. ]
 [0. ]
 [1.2]] 

Cost : 
 0.8000000000000003 



## Problem 2

We have five constarint equation, and four of them follow inequalities condition, Therefore we need four slack variable, let's call them $s_1,s_2,s_3,s_4$. Let's define the basic variale as $x_1,x_2,x_3$ for T,A and P. The LP that need to be solved is:<br><br>
$$-100x_1 + 0x_2 + 0x_3 + 1s_1 + 0s_2 + 0s_3 + 0s_4 = -5$$<br>
$$-2x_1 - 4.8x_2 - 19.7x_3 + 0s_1 + 1s_2 + 0s_3 + 0s_4 = -5.5$$<br>
$$ 2x_1 + 4.8x_2 + 19.7x_3 + 0s_1 + 0s_2 + 1s_3 + 0s_4 = 7$$<br>
$$-100x_1 - 125x_2 - 125x_3 + 0s_1 + 0s_2 + 0s_3 + 1s_4 = -5$$<br>
$$  x_1 +  x_2 +  x_3 + 0s_1 + 0s_2 + 0s_3 + 0s_4 = 1$$<br>

where $$ s_1 =  -5 - (-100x_1)$$<br>
$$ s_2 =  -5.5 - (-2x_1 - 4.8x_2 - 19.7x_3) $$<br>
$$ s_3 =  7 - (2x_1 + 4.8x_2 + 19.7x_3)$$<br>
$$ s_4 =  -115 - (-100x_1 - 125x_2 - 125x_3)$$<br>

In [377]:
A3 = np.array([[-100,  0,    0,    1,  0,  0,  0],
               [-2.0, -4.8, -19.7, 0,  1,  0,  0],
               [ 2.0,  4.8, 19.7, 0,  0,  1,   0],
               [-100, -125, -125,  0,  0,  0,  1],
               [ 1,    1,    1,    0,  0,  0,  0]])

C3 = np.array( [45,    30,   30,   0,  0,  0,  0]).reshape(-1,1)

b3 = np.array ([-5,-5.5,7,-115,1]).reshape(-1,1)

Initial_col = np.array([0,1,2,3,4])
Result3 = Solver(A3,C3,b3,Initial_col)

For 1th Iteration B :
 [[-100.     0.     0.     0.     0. ]
 [  -2.    -4.8  -19.7    0.     1. ]
 [   2.     4.8   19.7    0.     0. ]
 [-100.  -125.  -125.     1.     0. ]
 [   1.     1.     1.     0.     0. ]] 

For 1th Iteration N :
 [[0. 1.]
 [0. 0.]
 [1. 0.]
 [0. 0.]
 [0. 0.]] 

For 1th Iteration r :
 [-4.50401138e-16 -6.00000000e-01] 

For 2th Iteration B :
 [[-100.     0.     0.     0.     0. ]
 [  -2.    -4.8  -19.7    0.     0. ]
 [   2.     4.8   19.7    0.     1. ]
 [-100.  -125.  -125.     1.     0. ]
 [   1.     1.     1.     0.     0. ]] 

For 2th Iteration N :
 [[0. 1.]
 [1. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]] 

For 2th Iteration r :
 [-5.08541755e-17  1.50000000e-01] 

Value of X : 
 [[0.05      ]
 [0.89362416]
 [0.05637584]
 [0.        ]
 [0.        ]
 [1.5       ]
 [8.75      ]] 

Cost : 
 30.75 



C:\Users\18503\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
